# SK Connectors, functions, native functions and pipes

Learning objectives:

- Configuring and getting a kernel instance
- Configuring a kernel instance with a GPT connector
- Understand what is an SK function
- Adding SK functions to the kernel
- Understand the different pipe types:
  - Core
  - Native
  - Inline
  - Fixed
  - From file
- Calling SK functions using the kernel

## Load the required .NET Packages

In [ ]:
#r "nuget: Microsoft.SemanticKernel, 1.0.0-beta8"
#r "nuget: dotenv.net"

using System.Threading.Tasks;
using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.Plugins.Core;
using Microsoft.SemanticKernel.Connectors.AI.OpenAI;
using Microsoft.SemanticKernel.Connectors.AI.OpenAI.AzureSdk;
using dotenv.net;


## Load the OpenAI variables from environment variables or an .env file

In [ ]:
DotEnv.Load();
var deploymentName = Environment.GetEnvironmentVariable("GPT_OPENAI_DEPLOYMENT_NAME");
var endpoint = Environment.GetEnvironmentVariable("GPT_OPENAI_ENDPOINT");
var apiKey = Environment.GetEnvironmentVariable("GPT_OPENAI_KEY");
var deployment = $"Using deployment: {deploymentName} at: {endpoint} with key {apiKey.Substring(0, 5)}...";
deployment

## Get a kernel instance

In [ ]:
var kernel = new KernelBuilder()
            .WithAzureOpenAIChatCompletionService(deploymentName, endpoint, apiKey)                        
            .Build();

## Simple pipeline

### Import an available skill

In [ ]:
var funyParagraphFunctionDef = "Write a funny paragraph about streaming";
var fixedFunction = kernel.CreateSemanticFunction(funyParagraphFunctionDef,new OpenAIRequestSettings() { MaxTokens = 500, Temperature = 0.3, TopP = 1 });


In [ ]:
var roleDisplayed = false;

Console.WriteLine("\n===  Semantic Function - Streaming ===\n");


// Streaming can be of any type depending on the underlying service the function is using.
await foreach (var update in kernel.RunStreamingAsync<StreamingChatContent>(funyParagraphFunction))
{
    // You will be always able to know the type of the update by checking the Type property.
    if (!roleDisplayed && update.Role.HasValue)
    {
        Console.WriteLine($"Role: {update.Role}");
        roleDisplayed = true;
    }

    if (update.Content is { Length: > 0 })
    {
        Console.Write(update.Content);
    }
};

### Create two native skills

In [ ]:
// Create a new Skill
class ConsoleSkill
{
    [SKFunction, Description("Trim whitespace from the start and end of a string.")]
    public string Log(string input)
    {
        Console.WriteLine(input);
        return input;
    }
}
var cskill = kernel.ImportFunctions(new ConsoleSkill());

// SK Native Skill to replace acronyms
class AcronymExpander
{
    static Dictionary<string,string> dict = new Dictionary<string,string>(){
        {"AKS", "Azure Kubernetes Service (AKS)"},
        {"ACI", "Azure Container Instance (ACI)"},
        {"ACA", "Azure Container Applications (ACA)"},
    };

    [SKFunction, Description("Replace acronyms with expanded terminology.")]
    public string Expand(string input) 
    {
            foreach(var key in dict.Keys)
            {
                var isFound = input.ToLower().IndexOf(key.ToLower())>=0;
                if (isFound) {
                    input = input.Replace(key,dict[key],StringComparison.CurrentCultureIgnoreCase);
                }
            }
            return input;
    }
}
var acronymSkill = kernel.ImportFunctions(new AcronymExpander());

### Use skills in a pipeline

In [ ]:
var result = await kernel.RunAsync("    i n f i n i t e     s p a c e     ",
    cskill["Log"],
    text["TrimStart"],
    cskill["Log"],
    text["TrimEnd"],
    cskill["Log"],
    text["Uppercase"],
    cskill["Log"]);



var result1 = await kernel.RunAsync("What are the top issues in working with AKS and ACA?",acronymSkill["Expand"]);
result1

## Pipeline with completion

### Create an SK in-line function to give funny excuses

In [ ]:
const string FunctionDefinition = @"
Generate a creative reason or excuse for the given event.
Be creative and be funny. Let your imagination run wild.

Event: I am running late.
Excuse: I was being held ransom by giraffe gangsters.

Event: I haven't been to the gym for a year
Excuse: I've been too busy training my pet dragon.

Event: {{$input}}
";

var excuseFunction = kernel.CreateSemanticFunction(FunctionDefinition, new OpenAIRequestSettings() { MaxTokens = 500, Temperature = 0.3, TopP = 1 });

### Execute the excuses plugin

In [ ]:
var result = await kernel.RunAsync("I am running late.", excuseFunction);
Console.WriteLine(result);

### Pipe the excuses plugin and uppercase the output

In [ ]:
var result1 = await kernel.RunAsync("I missed your birthday",
    excuseFunction,
    text["Uppercase"]);

Console.WriteLine(result1);

## Imported SK function

### Import the summarizer function from a folder

In [ ]:
var sumSkill = kernel.ImportSemanticFunctionsFromDirectory(
            "./skills",
            "SummarizeSkill");

### Execute a prompt, Summarize the output and uppercase it

In [ ]:
var ask = @"Cryptocurrency, sometimes called crypto-currency or crypto, 
is any form of currency that exists digitally or virtually and uses cryptography to secure transactions. 
Cryptocurrencies don't have a central issuing or regulating authority, 
instead using a decentralized system to record transactions and issue new units.";

var result2 = await kernel.RunAsync(
    ask,
    sumSkill["Summarize"],
    text["Uppercase"]
);

Console.WriteLine(result2);